In [2]:
import pyrealsense2 as rs2
import open3d as o3d
import numpy as np
import time
from spygrt.stream import Recording, NoMoreFrames, DualRecording
from spygrt.tracker import Tracker
from spygrt.calibration import Calibrator
import importlib
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
#SET UP

rec1 = 'V:/VISION_Data/133/Recording_Session/121622061907_2023Mar23023414.bag'
rec2 = 'V:/VISION_Data/133/Recording_Session/123122061443_2023Mar23023414.bag'
cal1 = 'V:/VISION_Data/133/RS_Calibration/121622061907_2023Mar23015527.bag'
cal2 = 'V:/VISION_Data/133/RS_Calibration/123122061443_2023Mar23015527.bag'
opt1 = 'V:/VISION_Data/133/OptiTrack_Calibration_Square/121622061907_2023Mar23015642.bag'
opt2 = 'V:/VISION_Data/133/OptiTrack_Calibration_Square/123122061443_2023Mar23015642.bag'
d = DualRecording(rec1,rec2)
d.start_stream()
c = DualRecording(cal1,cal2)
c.start_stream()
calibs = (Calibrator(c.stream1), Calibrator(c.stream2))
o = DualRecording(opt1,opt2)
o.start_stream()

ERROR:root:Could not resolve the input for 'stream1'.


ValueError: Could not resolve the input for 'stream1'.

In [10]:
# CALIBRATION FOR ESTRO

for i in range (100):
    try:
        calibs[0].find_corners3d()
        calibs[1].find_corners3d()
    except NoMoreFrames:
        print("Reached the end of stream, used only the first " +str(i) + " frames for calibration.")
        break

#calibs[0].corners_scale()
#calibs[1].corners_scale()
calibs[0].align_cameras(calibs[1])
calibs[1].align_to_board(cal2 = calibs[0])
for cal in calibs:
    cal.write_cal()
    cal._pose = o3d.core.Tensor(np.identity(4), dtype=o3d.core.Dtype.Float32,device=cal._stream.pcd.device)
c.load_calibration()
for i in range (100):
    try:
        calibs[0].find_corners3d()
        calibs[1].find_corners3d()
    except NoMoreFrames:
        print("Reached the end of stream, used only the first " +str(i) + " frames for calibration.")
        break

#calibs[0].corners_scale()
#calibs[1].corners_scale()
calibs[0].align_cameras(calibs[1])
calibs[1].align_to_board(cal2 = calibs[0])
adj = o3d.core.Tensor(np.identity(4), dtype=o3d.core.Dtype.Float32, device=cal._pose.device)
adj[1][1]=-1
adj[0][0]=-1
adj[0][3] = -0.0215
adj[1][3] = -0.010
adj[2][3] = -0.130
calibs[0]._pose = adj@calibs[0].pose
calibs[1]._pose = adj@calibs[1].pose
for cal in calibs:
    cal.write_cal()

In [4]:
#t = Tracker(ref_surface=d.pcd)
data = []
import time
a = time.time()
for i in range(120000):
    
    try:
        t.get_motion(d.compute_pcd(True))
    except RuntimeError:
        continue
    except NoMoreFrames:
        np.savetxt('track_133.txt',data)
        break
    nt = t.t_guess.inv()
    data.append(np.concatenate(([d.timestamp], Tracker.compute_translation(nt), Tracker.compute_rotation(nt))))
    if i%100 == 0:
        print(i/100)
        np.savetxt('track_133.txt',data)
b = time.time()
b-a   

0.0
Timestamps:1679542454117.2952 1679542454234.3252
Timestamps:1679542455696.3303 1679542455846.9177
Timestamps:1679542455864.31 1679542455981.311
Timestamps:1679542456267.46 1679542456384.4866
Timestamps:1679542456401.8428 1679542456518.874
Timestamps:1679542457140.955 1679542457291.6006
Timestamps:1679542457140.955 1679542457291.6006
Timestamps:1679542457577.7039 1679542457425.9807
Timestamps:1679542457812.8752 1679542457694.7402
Timestamps:1679542457812.8752 1679542457694.7402
Timestamps:1679542458048.046 1679542457929.904
Timestamps:1679542458048.046 1679542457929.904
Timestamps:1679542458753.5527 1679542458601.7944
Timestamps:1679542458955.1238 1679542458836.953
Timestamps:1679542458955.1238 1679542458836.953
Timestamps:1679542458955.1238 1679542458836.953
Timestamps:1679542458988.719 1679542459105.704
Timestamps:1679542459055.9092 1679542459172.8916
Timestamps:1679542459055.9092 1679542459172.8916
Timestamps:1679542459828.6096 1679542459676.8323
Timestamps:1679542459862.2073 167

Timestamps:1679542497223.0457 1679542497339.1658
Timestamps:1679542497223.0457 1679542497339.1658
Timestamps:1679542497223.0457 1679542497339.1658
Timestamps:1679542497962.1826 1679542497843.122
Timestamps:1679542498096.5667 1679542498347.0847
Timestamps:1679542498298.142 1679542498414.2798
Timestamps:1679542498970.062 1679542499086.2295
Timestamps:1679542498970.062 1679542499086.2295
Timestamps:1679542498970.062 1679542499086.2295
Timestamps:1679542498970.062 1679542499086.2295
Timestamps:1679542500481.9158 1679542500295.6816
Timestamps:1679542500481.9158 1679542500295.6816
Timestamps:1679542500549.1064 1679542500396.4663
Timestamps:1679542500885.0588 1679542500766.0156
6.0
Timestamps:1679542501389.0198 1679542501269.9463
Timestamps:1679542501389.0198 1679542501269.9463
Timestamps:1679542502027.3738 1679542502177.043
Timestamps:1679542502027.3738 1679542502177.043
Timestamps:1679542502396.9573 1679542502613.799
Timestamps:1679542502900.9353 1679542503050.5618
Timestamps:1679542502900.

Timestamps:1679542531626.9353 1679542531742.5405
Timestamps:1679542531828.5137 1679542531944.1162
Timestamps:1679542531929.3027 1679542532112.0957
Timestamps:1679542531929.3027 1679542532112.0957
Timestamps:1679542531929.3027 1679542532112.0957
Timestamps:1679542532634.8098 1679542532784.0105
Timestamps:1679542532735.5957 1679542533153.5598
10.0
Timestamps:1679542534012.2224 1679542534127.85
Timestamps:1679542534012.2224 1679542534127.85
Timestamps:1679542534280.9866 1679542534127.85
Timestamps:1679542534280.9866 1679542534161.4473
Timestamps:1679542534717.7258 1679542534531.015
Timestamps:1679542534717.7258 1679542534531.015
Timestamps:1679542534717.7258 1679542534531.015
Timestamps:1679542534717.7258 1679542534531.015
Timestamps:1679542535288.8845 1679542535438.135
Timestamps:1679542535288.8845 1679542535438.135
Timestamps:1679542535591.2847 1679542535438.135
Timestamps:1679542535692.0852 1679542535841.296
Timestamps:1679542536095.276 1679542536210.8582
Timestamps:1679542536128.8718 

Timestamps:1679542560991.3472 1679542560837.4624
Timestamps:1679542560991.3472 1679542560837.4624
Timestamps:1679542560991.3472 1679542560837.4624
Timestamps:1679542560991.3472 1679542560837.4624
Timestamps:1679542560991.3472 1679542560837.4624
Timestamps:1679542560991.3472 1679542560837.4624
Timestamps:1679542560991.3472 1679542561207.0093
Timestamps:1679542560991.3472 1679542561207.0093
Timestamps:1679542560991.3472 1679542561207.0093
Timestamps:1679542560991.3472 1679542561207.0093
Timestamps:1679542561461.7087 1679542561710.935
Timestamps:1679542561596.0967 1679542561710.935
Timestamps:1679542561797.6785 1679542561912.5063
Timestamps:1679542561797.6785 1679542561912.5063
Timestamps:1679542561797.6785 1679542561912.5063
Timestamps:1679542561797.6785 1679542561912.5063
Timestamps:1679542562066.4539 1679542561912.5063
Timestamps:1679542562066.4539 1679542561912.5063
Timestamps:1679542562066.4539 1679542561912.5063
Timestamps:1679542562301.6409 1679542562416.4324
Timestamps:16795425623

Timestamps:1679542586055.3965 1679542585934.6226
Timestamps:1679542586391.3655 1679542586270.559
Timestamps:1679542586391.3655 1679542586270.559
Timestamps:1679542586391.3655 1679542586270.559
Timestamps:1679542586391.3655 1679542586270.559
Timestamps:1679542586391.3655 1679542586539.3066
Timestamps:1679542586391.3655 1679542586539.3066
Timestamps:1679542586391.3655 1679542586539.3066
Timestamps:1679542586391.3655 1679542586539.3066
Timestamps:1679542586391.3655 1679542586539.3066
Timestamps:1679542587466.4602 1679542587580.8406
Timestamps:1679542587869.6199 1679542588286.35
Timestamps:1679542587869.6199 1679542588286.35
Timestamps:1679542587869.6199 1679542588286.35
Timestamps:1679542587869.6199 1679542588286.35
Timestamps:1679542587869.6199 1679542588286.35
Timestamps:1679542587869.6199 1679542588286.35
Timestamps:1679542588911.156 1679542588790.26
Timestamps:1679542588911.156 1679542588790.26
Timestamps:1679542588911.156 1679542588790.26
Timestamps:1679542588911.156 1679542588790.26

...

Timestamps:1679543096270.5488 1679543096139.188
Timestamps:1679543096270.5488 1679543096139.188
Timestamps:1679543096270.5488 1679543096139.188
Timestamps:1679543096270.5488 1679543096139.188
Timestamps:1679543096270.5488 1679543096139.188
Timestamps:1679543096270.5488 1679543096139.188
Timestamps:1679543096270.5488 1679543096139.188
Timestamps:1679543096270.5488 1679543096139.188
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.5488 1679543097953.4062
Timestamps:1679543096270.548

78.0
Timestamps:1679543120863.8962 1679543120966.5823
Timestamps:1679543121368.364 1679543121470.4807
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:1679543121738.3372 1679543121504.0842
Timestamps:16795

Timestamps:1679543133061.0405 1679543133163.6191
Timestamps:1679543133296.1765 1679543133432.5754
Timestamps:1679543133296.1765 1679543133432.5754
Timestamps:1679543133665.6687 1679543133533.436
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134606.1902 1679543134205.1648
Timestamps:1679543134

Timestamps:1679543140955.9856 1679543141057.599
Timestamps:1679543140955.9856 1679543141057.599
Timestamps:1679543141728.9604 1679543141628.8625
Timestamps:1679543141728.9604 1679543141628.8625
Timestamps:1679543141728.9604 1679543141830.44
80.0
Timestamps:1679543142635.9587 1679543142737.6257
Timestamps:1679543142635.9587 1679543142737.6257


2321.35595536232

In [4]:
o.load_calibration(force=True)

(array([[ 0.73145878, -0.46030766,  0.50307554, -0.38821465],
        [-0.68160343, -0.51479334,  0.52000433, -0.41811669],
        [ 0.01961797, -0.72325975, -0.69029737,  0.48424962],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.75355726,  0.47636428, -0.4530215 ,  0.41423345],
        [ 0.6517849 , -0.4516702 ,  0.60923761, -0.44040293],
        [ 0.08560272, -0.75436801, -0.65084642,  0.49168023],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [7]:
t = Tracker(ref_surface=d.pcd)

[Open3D INFO] Picked point #194085 (0.05, -0.056, 0.13) to add in queue.
[Open3D INFO] Picked point #670412 (-0.08, 0.091, 0.13) to add in queue.


In [5]:
ot = Tracker(ref_surface=o.pcd)

[Open3D INFO] Picked point #973481 (0.013, -0.22, -0.11) to add in queue.
[Open3D INFO] Picked point #237553 (0.0081, -0.025, -0.1) to add in queue.
[Open3D INFO] Picked point #1285354 (0.16, -0.023, -0.098) to add in queue.


In [6]:
for cal in calibs:
    cal.write_cal()

In [ ]:
[Open3D INFO] Picked point #1061769 (-0.037, 0.21, 0.023) to add in queue.
[Open3D INFO] Picked point #220126 (-0.028, 0.015, 0.027) to add in queue.
[Open3D INFO] Picked point #1023532 (-0.18, 0.013, 0.029) to add in queue.